In [7]:
!pip show tensorflow

Name: tensorflow
Version: 2.15.0
Summary: TensorFlow is an open source machine learning framework for everyone.
Home-page: https://www.tensorflow.org/
Author: Google Inc.
Author-email: packages@tensorflow.org
License: Apache 2.0
Location: c:\users\yujus\anaconda3\envs\dl\lib\site-packages
Requires: tensorflow-intel
Required-by: tensorflow-decision-forests, tensorflow-serving-api, tensorflowjs


In [1]:
import numpy as np
import tensorflow as tf
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
import time
import keras


In [2]:
train_path = 'data/Master Folder/train'
validation_path = 'data/Master Folder/valid'

img_width = 224
img_height = 224

datagen = keras.preprocessing.image.ImageDataGenerator(
    #rescale = 1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    brightness_range=[0.4,0.6],
    zoom_range=0.3,
    horizontal_flip=True,
    vertical_flip=True
)
batch_size = 32

train_generator = datagen.flow_from_directory(
    train_path,
    target_size=(img_width, img_height),
    batch_size = batch_size,
    class_mode = 'categorical',
    color_mode = 'rgb',
    shuffle = True,
    seed = 42
)

validation_generator = datagen.flow_from_directory(
    validation_path,
    target_size=(img_width, img_height),
    batch_size = batch_size,
    class_mode = 'categorical',
    color_mode = 'rgb',
    shuffle = True,
    seed = 42
)


Found 1000 images belonging to 4 classes.
Found 36 images belonging to 4 classes.


In [3]:
input = tf.keras.layers.Input(shape=(224,224,3))
conv1 = tf.keras.layers.Conv2D(kernel_size=(3,3), padding="same", activation="relu", filters=64, input_shape=(224,224,3))(input)
#test dropout layers after activation layers
pool1 = tf.keras.layers.MaxPooling2D(pool_size=(2,2))(conv1)
bn1 = tf.keras.layers.BatchNormalization()(pool1)

conv2 = tf.keras.layers.Conv2D(kernel_size=(3,3), padding="same", activation="relu", filters=64)(bn1)
pool2 = tf.keras.layers.MaxPooling2D(pool_size=(2,2))(conv2)
bn2 = tf.keras.layers.BatchNormalization()(pool2)

conv3 = tf.keras.layers.Conv2D(kernel_size=(3,3), padding="same", activation="relu", filters=64)(bn2)
pool3 = tf.keras.layers.MaxPooling2D(pool_size=(2,2))(conv3)
bn3 = tf.keras.layers.BatchNormalization()(pool3)

flatten = tf.keras.layers.Flatten()(bn3)

#test dropout layers between dense layers
Dense1 = tf.keras.layers.Dense(512, activation='relu')(flatten)
Drop1 = tf.keras.layers.Dropout(.2)(Dense1)
#Dense2 = tf.keras.layers.Dense(512, activation='relu')(Drop1)
#Drop2 = tf.keras.layers.Dropout(.2)(Dense2)
#Dense3 = tf.keras.layers.Dense(256, activation='relu')(Drop2)
#Drop3 = tf.keras.layers.Dropout(.2)(Dense3)
#Dense4 = tf.keras.layers.Dense(256, activation='relu')(Drop3)
#Drop4 = tf.keras.layers.Dropout(.2)(Dense4)
Output = tf.keras.layers.Dense(4, activation='softmax')(Drop1)

base_model = tf.keras.applications.efficientnet.EfficientNetB5(include_top= False, weights= "imagenet", input_shape= (224,224,3), pooling= 'max')
base_model.trainable = False
transfer_model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.BatchNormalization(axis= -1, momentum= 0.99, epsilon= 0.001),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(128,kernel_regularizer= tf.keras.regularizers.l2(l= 0.016), activity_regularizer= tf.keras.regularizers.l1(0.006),
                bias_regularizer= tf.keras.regularizers.l1(0.006), activation='relu'),
    tf.keras.layers.Dropout(.45, seed=42),
    tf.keras.layers.Dense(4,activation='softmax')
])

model = tf.keras.Model(inputs=input, outputs=Output)
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=.001), loss=tf.keras.losses.CategoricalCrossentropy(), metrics=['accuracy'])
model.summary()

transfer_model.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=.001), loss=tf.keras.losses.CategoricalCrossentropy(), metrics=['accuracy'])
transfer_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 conv2d (Conv2D)             (None, 224, 224, 64)      1792      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 112, 112, 64)     0         
 )                                                               
                                                                 
 batch_normalization (BatchN  (None, 112, 112, 64)     256       
 ormalization)                                                   
                                                                 
 conv2d_1 (Conv2D)           (None, 112, 112, 64)      36928     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 56, 56, 64)       0     

In [4]:
callback = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=20, baseline=.1, verbose=1, restore_best_weights=True)
#load_model = keras.models.load_model("round2-3x3chess_eval.keras")
#load_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=.0005), loss=tf.keras.losses.MeanSquaredError(), metrics=[tf.keras.metrics.MeanAbsoluteError()])
history = transfer_model.fit(x=train_generator, epochs=100, shuffle=True, validation_data=(validation_generator), validation_freq=1, callbacks=callback)
model.save("petemo_model2.keras")

Epoch 1/100
32/32 [==============================] - 56s 948ms/step - loss: 4.2211 - accuracy: 0.4090 - val_loss: 4.3892 - val_accuracy: 0.2778
Epoch 2/100
32/32 [==============================] - 17s 521ms/step - loss: 3.3772 - accuracy: 0.4920 - val_loss: 3.4688 - val_accuracy: 0.4444
Epoch 3/100
32/32 [==============================] - 17s 512ms/step - loss: 2.8729 - accuracy: 0.5430 - val_loss: 2.9598 - val_accuracy: 0.3889
Epoch 4/100
32/32 [==============================] - 17s 526ms/step - loss: 2.4497 - accuracy: 0.6150 - val_loss: 2.6956 - val_accuracy: 0.5556
Epoch 5/100
32/32 [==============================] - 16s 506ms/step - loss: 2.1320 - accuracy: 0.6430 - val_loss: 2.5963 - val_accuracy: 0.4444
Epoch 6/100
32/32 [==============================] - 16s 508ms/step - loss: 1.9227 - accuracy: 0.6600 - val_loss: 2.2639 - val_accuracy: 0.5000
Epoch 7/100
32/32 [==============================] - 16s 499ms/step - loss: 1.6982 - accuracy: 0.6750 - val_loss: 2.1146 - val_accuracy:

In [57]:
from tensorflow.keras.preprocessing import image

test_path = 'data/Master Folder/test/Sad/018.jpg'
load_model = keras.models.load_model('petemo_model.keras')
img = image.load_img(test_path, target_size=(224, 224))
x=image.img_to_array(img)
x=np.expand_dims(x, axis=0)
images = np.vstack([x])

pred = load_model.predict(images, batch_size=10)
print(pred)
print(np.argmax(pred,axis=1))

1/1 [==============================] - 0s 135ms/step
[[2.4820996e-09 1.7755623e-07 5.9820577e-03 9.9401772e-01]]
[3]


In [1]:
import tensorflowjs as tfjs
load_model = keras.models.load_model('petemo_model.keras')
tfjs.converters.save_keras_model(load_model, petemo.json)

ModuleNotFoundError: No module named 'tensorflow.compat'